In [1]:
import requests_cache
from datetime import timedelta
# from mappings import geonames_countries
# from mappings import abstract_origins
from mappings import mapping

ROR_API_URL = "https://api.ror.org/organizations?affiliation="




def my_xml_escape(xmlstring):
    for key, value in mapping.items():
        xmlstring = xmlstring.replace(key, value)
    return xmlstring



# def my_xml_escape(xmlstring):
#     cases = [
#         ("&", "&amp;"),
#         ("<", "&lt;"),
#         (">", "&gt;"),
#         ("\"", "&quot;")
#     ]
#     for case in cases:
#         xmlstring = xmlstring.replace(case[0], case[1]) 
#     return xmlstring

def replace_encodings(text):
    cases = [
        # ("&amp;", "&"),
        # ("&lt;", "<"),
        # ("&gt;", ">")
        # ("^D$e", "€"),
        # ("^D#&gt;", "≥"),
        # ("^DEL", "…"),
        # ("^DIF", "∞"),
        # ("^D#=D", "≠"),
        ("^DDS", "-")
        # lots more where this came from...
    ]
    for case in cases:
        text = text.replace(case[0], case[1])   
    return text


urls_expire_after = {
    # Custom cache duration per url, 0 means "don't cache"
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/09183&lang=de': 0,
    # f'{SKOSMOS_URL}/rest/v1/label?uri=https%3A//w3id.org/zpid/vocabs/terms/': 0,
}

session = requests_cache.CachedSession(
    ".cache/requests",
    allowable_codes=[200, 404],
    expire_after=timedelta(days=30),
    urls_expire_after=urls_expire_after,
)

def get_ror_id_from_api(affiliation_string):
    # this function takes a string with an affiliation name and returns the ror id for that affiliation from the ror api
    # clean the string to make sure things like "^DDS" are replaced:
    affiliation_string = my_xml_escape(affiliation_string)
    #replace_encodings(affiliation_string)
    ror_api_url = ROR_API_URL + affiliation_string
    # make a request to the ror api:
    # ror_api_request = requests.get(ror_api_url)
    # make request to api with caching:
    ror_api_request = session.get(
            ror_api_url, timeout=20
    )
    # if the request was successful, get the json response:
    if ror_api_request.status_code == 200:
        ror_api_response = ror_api_request.json()
        # check if the response has any hits:
        if len(ror_api_response["items"]) > 0:
            # if so, get the item with a key value pair of "chosen" and "true" and return its id:
            for item in ror_api_response["items"]:
                if item["chosen"] == True:
                    return item["organization"]["id"]
        else:
            return None
    else:
        return None

In [2]:
# here is a list of affiliation strings to go through:
affiliation_strings = [
    "Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Sektion Medizinische Psychologie, Universitätsklinikum Ulm",
    "Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm",
    "Psychology School, Hochschule Fresenius ^DDS University of Applied Sciences, Düsseldorf",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Fakultät Medizin, MSH Medical School Hamburg ^DDS University of Applied Sciences and Medical University, Hamburg",
    "Department of Child and Adolescent Psychiatry, Psychosomatics and Psychotherapy; LVR Klinikum Essen; University Hospital Essen; University of Duisburg-Essen; Essen",
    "Child and Adolescent Psychiatry/Psychology, Erasmus Medical Center Rotterdam"
]

# use the function to get the ror id for each affiliation string:

for affiliation_string in affiliation_strings:
    print(my_xml_escape(affiliation_string) + ": " + str(get_ror_id_from_api(affiliation_string)))

print(my_xml_escape("Stimulus ^DDS non psychological interference ^DD analogy"))

# print(get_ror_id_from_api("Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm"))

Klinik für Frauenheilkunde und Geburtshilfe, Universitätsklinikum Ulm: https://ror.org/05emabm63
Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm: https://ror.org/05emabm63
Sektion Medizinische Psychologie, Universitätsklinikum Ulm: https://ror.org/05emabm63
Klinik für Psychososmatische Medizin und Psychotherapie, Universitätsklinikum Ulm: https://ror.org/05emabm63
Psychology School, Hochschule Fresenius - University of Applied Sciences, Düsseldorf: https://ror.org/03hj50651
Fakultät Medizin, MSH Medical School Hamburg - University of Applied Sciences and Medical University, Hamburg: https://ror.org/006thab72
Fakultät Medizin, MSH Medical School Hamburg - University of Applied Sciences and Medical University, Hamburg: https://ror.org/006thab72
Department of Child and Adolescent Psychiatry, Psychosomatics and Psychotherapy; LVR Klinikum Essen; University Hospital Essen; University of Duisburg-Essen; Essen: None
Child and Adolescent Psychiatry/Psychology,